<a href="https://colab.research.google.com/github/jessycalais/Projetos_AnaliseDeDados/blob/main/Projeto02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I) Conexão com a API do **BigQuery**

In [ ]:
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
print('Authenticated')

Authenticated


II) A tabela utilizada neste projeto está disponível no conjunto de dados públicos da **Base dos Dados**.
> Você pode verificar mais informações em:   
> https://basedosdados.org/dataset/2a543ad8-3cdb-4047-9498-efe7fb8ed697?table=df7cf198-4889-4baf-bb77-4e0e28eb90ca

In [ ]:
client = bigquery.Client(project='formal-chassis-396823')

dados_gerais = client.query('''
                            SELECT DISTINCT
                              m.id_municipio,
                              m.ano,
                              m.sigla_uf,
                              CASE
                                WHEN m.sigla_uf IN ('AM', 'PA', 'RR', 'RO', 'AP', 'AC', 'TO') THEN 'Norte'
                                WHEN m.sigla_uf IN ('PI', 'MA', 'PE', 'RN', 'PB', 'CE', 'BA', 'AL', 'SE') THEN 'Nordeste'
                                WHEN m.sigla_uf IN ('SP', 'MG', 'RJ', 'ES') THEN 'Sudeste'
                                WHEN m.sigla_uf IN ('PR', 'SC', 'RS') THEN 'Sul'
                                ELSE 'Centro-Oeste'
                              END
                              AS regiao,
                              m.populacao_urbana,                  --	População urbana do município do ano de referência (Fonte: IBGE)
                              m.populacao_urbana_residente_agua,   -- População urbana residente do(s) município(s) com abastecimento de água
                              m.populacao_urbana_residente_esgoto, --	População urbana residente do(s) município(s) com esgotamento sanitário
                              m.populacao_atendida_agua,           -- População total atendida com abastecimento de água
                              m.populacao_atentida_esgoto,         -- População total atendida com esgotamento sanitário
                              p.prestador,                         -- Nome do prestador de serviço
                              p.tipo_servico                       -- Tipo do prestador de serviço
                            FROM
                              `basedosdados.br_mdr_snis.municipio_agua_esgoto` AS m
                            INNER JOIN
                              `basedosdados.br_mdr_snis.prestador_agua_esgoto` AS p
                            ON
                              m.id_municipio = p.id_municipio
                            WHERE
                              m.ano BETWEEN 2019 AND 2021
                            ORDER BY
                              m.sigla_uf,
                              ano
                            ''').to_dataframe()

In [ ]:
dados_gerais.head()

,id_municipio,ano,sigla_uf,regiao,populacao_urbana,populacao_urbana_residente_agua,populacao_urbana_residente_esgoto,populacao_atendida_agua,populacao_atentida_esgoto,prestador,tipo_servico
0,1200013,2019,AC,Norte,7198,7198,<NA>,5263,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua
1,1200054,2019,AC,Norte,4520,4520,<NA>,5165,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua
2,1200104,2019,AC,Norte,17508,17508,<NA>,16468,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua
3,1200104,2019,AC,Norte,17508,17508,<NA>,16468,<NA>,Prefeitura Municipal de Brasileia,Esgotos
4,1200138,2019,AC,Norte,4476,4476,<NA>,4865,<NA>,Prefeitura Municipal de Bujari,Esgotos


In [ ]:
dados_gerais.dtypes

id_municipio                         object
ano                                   Int64
sigla_uf                             object
regiao                               object
populacao_urbana                      Int64
populacao_urbana_residente_agua       Int64
populacao_urbana_residente_esgoto     Int64
populacao_atendida_agua               Int64
populacao_atentida_esgoto             Int64
prestador                            object
tipo_servico                         object
dtype: object

III) Leitura do arquivo `.xlsx` que contém os nomes dos municípios por ID do IBGE.

> Disponível em: https://www.ibge.gov.br/explica/codigos-dos-municipios.php

In [ ]:
# imports e leitura de arquivo ".xlsx"
import pandas as pd

nome_municipios_ibge = pd.read_excel('/content/drive/MyDrive/RELATORIO_DTB_BRASIL_MUNICIPIO.xlsx',
                                    skiprows=range(0, 6),
                                    usecols=['Nome_Município', 'Código Município Completo'],
                                    dtype={'Nome_Município':'str', 'Código Município Completo':'str'})

In [ ]:
nome_municipios_ibge.dtypes

Código Município Completo    object
Nome_Município               object
dtype: object

In [ ]:
nome_municipios_ibge.rename(columns={'Nome_Município':'municipio', 'Código Município Completo':'id_municipio_ibge'}, inplace=True)
nome_municipios_ibge

,id_municipio_ibge,municipio
0,1100015,Alta Floresta D'Oeste
1,1100379,Alto Alegre dos Parecis
2,1100403,Alto Paraíso
3,1100346,Alvorada D'Oeste
4,1100023,Ariquemes
...,...,...
5565,5222005,Vianópolis
5566,5222054,Vicentinópolis
5567,5222203,Vila Boa
5568,5222302,Vila Propício


IV) Utilizando o método `.merge` da biblioteca Pandas para fazer um INNER JOIN com os dados obtidos via BigQuery a fim de levar os nomes do município para a base extraída via SQL.

In [ ]:
dados_pre_analise = dados_gerais.merge(nome_municipios_ibge, how='inner', left_on='id_municipio', right_on='id_municipio_ibge')
dados_pre_analise.head()

,id_municipio,ano,sigla_uf,regiao,populacao_urbana,populacao_urbana_residente_agua,populacao_urbana_residente_esgoto,populacao_atendida_agua,populacao_atentida_esgoto,prestador,tipo_servico,id_municipio_ibge,municipio
0,1200013,2019,AC,Norte,7198,7198,<NA>,5263,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua,1200013,Acrelândia
1,1200013,2019,AC,Norte,7198,7198,<NA>,5263,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua e Esgoto,1200013,Acrelândia
2,1200013,2020,AC,Norte,7309,7309,<NA>,5198,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua,1200013,Acrelândia
3,1200013,2020,AC,Norte,7309,7309,<NA>,5198,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua e Esgoto,1200013,Acrelândia
4,1200013,2021,AC,Norte,7418,7418,<NA>,5589,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua,1200013,Acrelândia


V) Organizando as colunas para análise

In [ ]:
dados_para_analise = dados_pre_analise \
                                      .sort_values(by=['sigla_uf', ]) \
                                      .rename(columns={'populacao_atentida_esgoto':'populacao_atendida_esgoto'}) \
                                      .drop(['id_municipio_ibge', 'id_municipio'], axis=1) \
                                      .reset_index(drop=True)

In [ ]:
dados_para_analise.insert(3, 'municipio', dados_para_analise.pop('municipio'))
dados_para_analise.head()

,ano,sigla_uf,regiao,municipio,populacao_urbana,populacao_urbana_residente_agua,populacao_urbana_residente_esgoto,populacao_atendida_agua,populacao_atendida_esgoto,prestador,tipo_servico
0,2019,AC,Norte,Acrelândia,7198,7198,<NA>,5263,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua
1,2021,AC,Norte,Plácido de Castro,12154,12154,12154,8764,1783,Prefeitura Municipal de Placido de Castro,Esgoto
2,2019,AC,Norte,Porto Acre,2465,2465,<NA>,3915,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua
3,2019,AC,Norte,Porto Acre,2465,2465,<NA>,3915,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua e Esgoto
4,2020,AC,Norte,Porto Acre,2507,2507,<NA>,3900,<NA>,Departamento Estadual de Pavimentacao e Saneam...,Agua
